In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import mysql.connector as mysql
import json
import io
import datetime
import os
from PIL import Image
from folium.plugins import FastMarkerCluster
from selenium import webdriver
from time import sleep

path = os.getcwd()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(path+'\\chromedriver.exe', options=options)


C:\Users\DANILO~1.MAR\AppData\Local\Temp/ipykernel_11852/38591692.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path+'\\chromedriver.exe', options=options)


In [13]:
# consulta do mapa de escolas
geo_escolas = '''
        select 
	concat(tpi.tpi_descricao, ' ', ins.ins_nome) as nome_escola
    , aep.aep_descricao as etapa_escolar
	, ende.enc_latitude as latitude
	, ende.enc_longitude as longitude
    , ins.ins_id
from
	academico.aep_anoEscolarPadrao as aep
	join
		academico.ins_instituicao as ins
			on ins.ins_id = aep.ins_id
	join
		academico.pel_periodoLetivo as pel
			on pel.pel_id = aep.pel_id
	join
		academico.tpi_tipoInstituicao as tpi
			on tpi.tpi_id = ins.tpi_id
	join
		academico.enc_enderecoinstituicao as ende
			on ins.ins_id = ende.ins_id
where
	pel.pel_anoVigente = 2022
    and aep.aep_descricao in('MATERNAL', 'MINI MATERNAL', 'JARDIM I', 'JARDIM II', 'BERÇÁRIO');
            '''

alunos = '''
    select DISTINCT
	ins.ses_descricao as etapa_escolar
	, acadEnd.end_latitude as latitude
	, acadEnd.end_longitude as longitude
	, ins.ins_id 
from
	ssv.ins_inscricao as ins
	join
		ssv.pss_pessoa as pss
			on pss.pss_id = ins.pss_id_crianca
	join
		ssv.end_endereco as ssvEnd
			on ssvEnd.end_id = pss.end_id
	left join
		academico.end_endereco as acadEnd
			on ssvEnd.end_idAcademico = acadEnd.end_id
	join
		ssv.cfv_configuracaoVagas as cfg
			on cfg.cfg_id = ins.cfg_id
where
	cfg.cfg_anoMatricula = 2022
	and acadEnd.end_latitude is not null
;
'''

opcao_escola = '''
    select
	ope.ins_id
	, ope.ope_instituicaoId 
from
	ssv.ope_opcaoEscola ope
	join
		(select ins_id, max(ope_id) as ope_id from ssv.ope_opcaoEscola where ope_numeroOpcao = 1 and ope_ativo = 1 group by ins_id) opc
			on opc.ope_id = ope.ope_id
where
	ope.ope_numeroOpcao = 1
	;
'''

qtd_vagas = '''
	select
    ins.ins_id
    , aep.aep_descricao
    , sum(tui.tui_maxAlunos) as qtd_vagas
from
    academico.tui_turmaInstituicao as tui
    join
        academico.aep_anoEscolarPadrao as aep
        on aep.aep_id = tui.aep_id
    join
        academico.ins_instituicao as ins
        on ins.ins_id = aep.ins_id
    join
        academico.pel_periodoLetivo as pel
        on pel.pel_id = aep.pel_id
where
    pel.pel_anoVigente = 2022
    and aep.aep_descricao in('MATERNAL', 'MINI MATERNAL', 'JARDIM I', 'JARDIM II', 'BERÇÁRIO')
group by
    ins.ins_id
    , aep.aep_descricao;
'''

hst = "52.15.209.136"
usr = "bi"
passwd = 'fRQJILefuz7'
    
db = mysql.connect(
    host = hst,
    user = usr,
    passwd = passwd,
    charset ='utf8',
    )

cursor = db.cursor()
cursor.execute(geo_escolas)
data_escola = cursor.fetchall()
cursor.execute(alunos)
data_alunos = cursor.fetchall()
cursor.execute(opcao_escola)
data_opcao = cursor.fetchall()
cursor.execute(qtd_vagas)
data_qtd = cursor.fetchall()

# df escola
df_escola = []
n_escola = 0
for i in data_escola:
    df_escola.append({"escola": i[0], "latitude_escola": i[2], "longitude_escola": i[3],
        'id_escola': i[4]})
    n_escola += 1
df_escola = json.dumps(df_escola)
df_escola = pd.read_json(df_escola)

# df aluno
df_aluno = []
n_aluno = 0
for i in data_alunos:
    df_aluno.append({"etapa": i[0], "latitude": i[1], "longitude": i[2], 'id_inscricao': i[3],
        'id_inscricao_controle': i[3]})
    n_aluno += 1
df_aluno = json.dumps(df_aluno)
df_aluno = pd.read_json(df_aluno)

# df opcao
df_opcao = []
for i in data_opcao:
    df_opcao.append({'id_inscricao' : i[0], 'id_escola' : i[1]})
df_opcao = json.dumps(df_opcao)
df_opcao = pd.read_json(df_opcao)

# df qtde vagas
df_qtde = []
for i in data_qtd:
	df_qtde.append({'id_escola': i[0], 'etapa': i[1], 'qde_vagas': i[2]})
df_qtde = json.dumps(df_qtde)
df_qtde = pd.read_json(df_qtde)

# join dos alunos com opção de escola
df_pars = df_aluno.set_index('id_inscricao').join(df_opcao.set_index('id_inscricao'))

# join dos alunos e opções com quantidade de vagas
df_pars = pd.merge(df_pars, df_qtde, on=['id_escola', 'etapa'], how='left')

# join dos alunos com escolas
df = df_pars.set_index('id_escola').join(df_escola.set_index('id_escola'), on='id_escola')

# removendo duplicados
df = df.drop_duplicates()
df = df.dropna()
#cursor.close()


In [14]:
# df mapa escola
mapa_escola = df[['escola', 'latitude_escola', 'longitude_escola']].drop_duplicates()
n_escola = mapa_escola.count()[0]

latitude_escola = mapa_escola.latitude_escola.values
longitude_escola = mapa_escola.longitude_escola.values
escola = mapa_escola.escola.values


In [15]:
# df mapa aluno

mapa_alunos = df[['id_inscricao_controle', 'latitude', 'longitude']].drop_duplicates()
n_aluno = mapa_alunos.count()[0]

latitude_aluno = mapa_alunos.latitude.values
longitude_aluno = mapa_alunos.longitude.values
aluno = mapa_alunos.id_inscricao_controle.values


In [16]:
# montagem do mapa de escolas
ts = folium.Map(location=[-23.61957807914073, -46.78906836802145],
                 zoom_start=14, zoom_control=False)

for latitude, longitude, escola in zip(latitude_escola, longitude_escola, escola):
    folium.Marker(location=[float(latitude), float(longitude)], popup=escola, icon=folium.Icon(color='blue',
                                                                                              #,icon_color='blue'
                                                                                               )).add_to(ts)
ts.save('escolas.html')
driver.get('file://'+path+'/escolas.html')
sleep(4)
driver.save_screenshot('escolas.png')

True

In [17]:
# montagem do mapa com alunos
latitude_escola = mapa_escola.latitude_escola.values
longitude_escola = mapa_escola.longitude_escola.values
escola = mapa_escola.escola.values


ts_aluno = folium.Map(location=[-23.61957807914073, -46.78906836802145],
                 zoom_start=14, zoom_control=False)

for latitude, longitude, escola in zip(latitude_escola, longitude_escola, escola):
    folium.Marker(location=[float(latitude), float(longitude)], popup=escola, icon=folium.Icon(color='blue',
                                                                                              #,icon_color='blue'
                                                                                              )).add_to(ts_aluno)

ts_aluno.add_child(FastMarkerCluster(mapa_alunos[['latitude', 'longitude']]))

ts_aluno.save('mapa_completo.html')
driver.get('file://'+path+'/mapa_completo.html')
sleep(4)
driver.save_screenshot('mapa_completo.png')

True

<img  width="10%" src="planneta.png">

# ANÁLISE DEMOGRÁFICA DE SOLICITAÇÃO DE VAGAS EM TABOÃO DA SERRA

Este estudo visa demonstrar a distribuição das solicitações de vagas no município de Taboão da Serra, tendo como indicadores a **geolocalização dos alunos**, **geolocalização das escolas**, **etapa de ensino das vagas** e **quantitativo de solicitações**.

O estudo foi feito utilizando toda a base de dados contida no sistema **Gestão Fácil** tendo como foco as etapas:

* BERÇÁRIO
* MINI MATERNAL
* MATERNAL
* JARDIM I
* JARDIM II

## Desenvolvimento

Para iniciar o estudo, foi plotado no mapa, todas as instituições ativas no município para se ter uma visão macro da distribuição das escolas.

In [18]:
print(f"Ao todo, o município possúi {n_escola} escolas ativas para o ano de {datetime.datetime.today().year + 1}, que compreendem as etapas de ensino analisadas.")

Ao todo, o município possúi 47 escolas ativas para o ano de 2022, que compreendem as etapas de ensino analisadas.


<center><img width="80%" src="escolas.png"></center>

Todas as escolas estão bem distribuídas ao longo do município o que faz com que seja necessário verificar a distribuição das solicitações de vaga de forma global.

Para que a análise possa ser visualmente interpretada, foi gerado um mapa que mostra a distribuição das solicitações em meio às escolas.

In [19]:
print(f"Ao todo, o município possui {n_aluno} solicitações de vagas até o momento deste estudo.")

Ao todo, o município possui 3531 solicitações de vagas até o momento deste estudo.


<center><img width="80%" src="mapa_completo.png"></center>

> É possível observar que a relação da distribuição de escolas e solicitações de vagas no município apresenta equilíbrio dentro da visão global.

A partir da distribuição global, uma análise mais detalhada por etapa de ensino seguirá.

### Distribuição de vagas para Berçário

O primeiro ponto a se observar, é a relação entre escolas que disponibilizam vagas para berçário e alunos que solicitam vagas nesta etapa de ensino.

In [20]:
df_bercario_escola = df.set_index('etapa')
df_bercario_escola = df_bercario_escola.loc['BERÇÁRIO']

df_bercario_escola = df_bercario_escola[['escola', 'latitude_escola', 'longitude_escola']]

df_bercario_escola = df_bercario_escola.drop_duplicates()

bercario_lat = df_bercario_escola.latitude_escola.values
bercario_lon = df_bercario_escola.longitude_escola.values
bercario_esc = df_bercario_escola.escola.values

df_bercario_aluno = df.set_index('etapa')
df_bercario_aluno = df_bercario_aluno.loc['BERÇÁRIO']

ts_bercario = folium.Map(location=[-23.61957807914073, -46.78906836802145],
                 zoom_start=14, zoom_control=False)


for latitude, longitude, escola in zip(bercario_lat, bercario_lon, bercario_esc):
    folium.Marker(location=[float(latitude), float(longitude)], popup=escola, icon=folium.Icon(color='blue',
                                                                                              #,icon_color='blue'
                                                                                              )).add_to(ts_bercario)
    
ts_bercario.add_child(FastMarkerCluster(df_bercario_aluno[['latitude', 'longitude']]))

folium.vector_layers.CircleMarker(
    location=[float('-23.628995357229115'), float('-46.80304992140114')],
    radius=180,
    color='#a62e2e',
    fill=True,
    fill_color='#a62e2e'
).add_to(ts_bercario)


folium.vector_layers.CircleMarker(
    location=[float('-23.623286580850486'), float('-46.78112923285396')],
    radius=80,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(ts_bercario)




ts_bercario

#ts_bercario.save('mapa_bercario.html')
#driver.get('file://'+path+'/mapa_bercario.html')
#sleep(4)
#driver.save_screenshot('mapa_bercario.png')

<center><img width="80%" src="C:\\Users\\danilo.mario\\Desktop\\estudo\\mapa_bercario.png"></center>

<center><img width="80%" src="mapa_bercario.png"></center>